<a href="https://colab.research.google.com/github/macorony/ConvNet_Examples/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

credit resource: https://www.youtube.com/watch?v=uQc4Fs7yx5I

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim

In [2]:
# define convolution block with different kernels
class conv_block(nn.Module):
  def __init__(self, in_channel, out_channel, **kwargs):
    super(conv_block, self).__init__()
    self.relu = nn.ReLU()
    self.conv = nn.Conv2d(in_channel, out_channel, **kwargs)
    self.batchnorm = nn.BatchNorm2d(out_channel)
  def forward(self, x):
    return self.relu(self.batchnorm(self.conv(x)))

In [3]:
# define inception block
class Inception_block(nn.Module):
  def __init__(self, in_channels, out_1x1, out_3x3_reduce, out_3x3, out_5x5_reduce, out_5x5, out_pool):
    super(Inception_block, self).__init__()
    # 1x1 convolution
    self.branch1 = nn.Conv2d(in_channels, out_1x1, kernel_size=1)
    # 1x1 -> 3x3 convolution
    self.branch2 = nn.Sequential(
        nn.Conv2d(in_channels, out_3x3_reduce, kernel_size=1),
        nn.Conv2d(out_3x3_reduce, out_3x3, kernel_size=3, padding=1)
    )
    # 1x1 -> 5x5 convolution
    self.branch3 = nn.Sequential(
        nn.Conv2d(in_channels, out_5x5_reduce, kernel_size=1),
        nn.Conv2d(out_5x5_reduce, out_5x5, kernel_size=5, padding=2)
    )
    # 3x3 max pooling -> 1x1 convolution
    self.branch4 = nn.Sequential(
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        nn.Conv2d(in_channels, out_pool, kernel_size=1)
    )

  def forward(self, x):
    branch1 = self.branch1(x)
    branch2 = self.branch2(x)
    branch3 = self.branch3(x)
    branch4 = self.branch4(x)

    outputs = [branch1, branch2, branch3, branch4]
    return torch.cat(outputs, 1)

In [4]:
# define GoogLeNet
class GoogLeNet(nn.Module):
  def __init__(self, num_classes=100):
    super(GoogLeNet, self).__init__()
    # initial layer
    self.pre_layers = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
        nn.MaxPool2d(3, stride=2, padding=1),
        nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
        nn.MaxPool2d(3, stride=2, padding=1)
    )
    # inception block
    self.inception3a = Inception_block(192, 64, 96, 128, 16, 32, 32)
    self.inception3b = Inception_block(256, 128, 128, 192, 32,96, 64)
    self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

    self.inception4a = Inception_block(480, 192, 96, 208, 16, 48, 64)
    self.inception4b = Inception_block(512, 160, 112, 224, 24, 64, 64)
    self.inception4c = Inception_block(512, 128, 128, 256, 24, 64, 64)
    self.inception4d = Inception_block(512, 112, 144, 288, 32, 64, 64)
    self.inception4e = Inception_block(528, 256, 160, 320, 32, 128, 128)

    self.inception5a = Inception_block(832, 256, 160, 320, 32, 128, 128)
    self.inception5b = Inception_block(832, 384, 192, 384, 48, 128, 128)

    # final layers
    self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
    self.dropout = nn.Dropout(0.5)
    self.fc = nn.Linear(1024, num_classes)

  def forward(self, x):
    x = self.pre_layers(x)
    x = self.inception3a(x)
    x = self.inception3b(x)
    x = self.maxpool(x)

    x = self.inception4a(x)
    x = self.inception4b(x)
    x = self.inception4c(x)
    x = self.inception4d(x)
    x = self.inception4e(x)
    x = self.maxpool(x)

    x = self.inception5a(x)
    x = self.inception5b(x)

    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.dropout(x)
    x = self.fc(x)

    return x

In [5]:
transform = transforms.Compose(
    [transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize((0.1307),(0.3081))
    ]
    )

In [6]:
train_set = torchvision.datasets.FashionMNIST('./fashion', train=True, transform=transform, download=True)
test_set = torchvision.datasets.FashionMNIST('./fashion', train=False, transform=transform, download=True)

In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=10, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=True, num_workers=2)

In [8]:
model = GoogLeNet(num_classes=10)

In [9]:
loss_fc = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [10]:
def model_training(loader, nnModel, EPOCH, optimizer, loss_function):
  nnModel.train()
  for epoch in range(EPOCH):
    print(f'Start to train the epoch {epoch+1}.')
    running_loss = 0
    for i, data in enumerate(loader):
      inputs, labels = data
      optimizer.zero_grad()
      outputs = nnModel(inputs)
      loss = loss_function(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss
      if i % 1000 == 999:
        print(f"At {i+1}th batch the loss is {running_loss}.")
        running_loss = 0
  print('Finish the training.')